

In **Part 1**, we will be performing Data Collection and Pre-Processing from the ChEMBL Database.

Note for this Concised Version:
* Redundant code cells were deleted.
* Code cells for saving files to Google Drive has been deleted.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.


## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [2]:
! pip install chembl_webresource_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.2 MB/s eta 0:00:00


## **Importing libraries**

In [3]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for Tau-tublin Kinases**

In [8]:
# Target search for Tau-tubulin Kinases
target = new_client.target
target_query = target.search('CHEMBL1926492')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,Tau-tubulin kinase 1,18.0,False,CHEMBL1926492,"[{'accession': 'Q5TCY1', 'component_descriptio...",SINGLE PROTEIN,9606


### **Select and retrieve bioactivity data for *TTBK1* (first entry)**

We will assign the first (which corresponds to the target protein, *TTKB1*) to the ***selected_target*** variable

In [9]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL1926492'

Here, we will retrieve only bioactivity data for *TTBK1* (CHEMBL1926492) that are reported as pChEMBL values.

In [10]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [11]:
df = pd.DataFrame.from_dict(res)

In [12]:
df

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,16432039,[],CHEMBL3750193,Inhibition of human TTBK1 using MBP substrate,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
1,None,None,18934333,[],CHEMBL4326705,Inhibition of TTBK1 (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,120.0
2,None,Not Determined,18939146,[],CHEMBL4328642,Inhibition of human TTBK1 using MBP as substra...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,None,None,None,None
3,None,None,18939147,[],CHEMBL4328642,Inhibition of human TTBK1 using MBP as substra...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,10'-5M,UO_0000065,None,4.72
4,None,None,19148080,[],CHEMBL4379476,Inhibition of recombinant human TTBK1 (1 to 47...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011527,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,1.85
110,None,None,25011528,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
111,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011529,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,3.54
112,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011530,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,377.0


Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [13]:
df.to_csv('TTBK1_01_bioactivity_data_raw.csv', index=False)

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [14]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

/tmp/ipython-input-3852201246.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[df.canonical_smiles.notna()]


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,16432039,[],CHEMBL3750193,Inhibition of human TTBK1 using MBP substrate,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
1,None,None,18934333,[],CHEMBL4326705,Inhibition of TTBK1 (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,120.0
3,None,None,18939147,[],CHEMBL4328642,Inhibition of human TTBK1 using MBP as substra...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,10'-5M,UO_0000065,None,4.72
4,None,None,19148080,[],CHEMBL4379476,Inhibition of recombinant human TTBK1 (1 to 47...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
5,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24884477,[],CHEMBL5156814,Inhibition of TTBK1 (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011527,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,1.85
110,None,None,25011528,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
111,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011529,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,3.54
112,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011530,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236877,Inhibition of TTBK1 in human HEK293T cells inc...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,377.0


In [15]:
len(df2.canonical_smiles.unique())

87

In [16]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,16432039,[],CHEMBL3750193,Inhibition of human TTBK1 using MBP substrate,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
1,None,None,18934333,[],CHEMBL4326705,Inhibition of TTBK1 (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,120.0
3,None,None,18939147,[],CHEMBL4328642,Inhibition of human TTBK1 using MBP as substra...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,10'-5M,UO_0000065,None,4.72
4,None,None,19148080,[],CHEMBL4379476,Inhibition of recombinant human TTBK1 (1 to 47...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,uM,UO_0000065,None,10.0
5,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24884477,[],CHEMBL5156814,Inhibition of TTBK1 (unknown origin),B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011503,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236876,Inhibition of GST-tagged TTBK1 (1 to 421) (unk...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,199.0
86,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011504,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236876,Inhibition of GST-tagged TTBK1 (1 to 421) (unk...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,1022.0
87,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011505,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236876,Inhibition of GST-tagged TTBK1 (1 to 421) (unk...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,185.0
88,"{'action_type': 'INHIBITOR', 'description': 'N...",None,25011506,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5236876,Inhibition of GST-tagged TTBK1 (1 to 421) (unk...,B,None,None,BAO_0000190,...,Homo sapiens,Tau-tubulin kinase 1,9606,None,None,IC50,nM,UO_0000065,None,352.0


## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [17]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL3745885,Cn1c(=O)c(S(=O)(=O)c2ccc(F)cc2F)cc2cnc(Nc3ccc4...,10000.0
1,CHEMBL4528881,CC1(N)CCC(Nc2ccn3ncnc(Nc4cc(O)cc(Br)c4)c23)CC1,120.0
3,CHEMBL278041,Oc1ccc(-c2nc(-c3ccc(F)cc3)c(-c3ccncc3)[nH]2)cc1,47200.0
4,CHEMBL4438748,CN(C)c1ccc(C(=O)Nc2cccc(NC(=O)COc3ccc4c(=O)cco...,10000.0
5,CHEMBL5193906,CC1(N)CCN(Cc2ccn3ncnc(Nc4cc(O)cc(Br)c4)c23)CC1,120.0
...,...,...,...
85,CHEMBL2334586,CC(C)(O)C#Cc1ccc2[nH]c3c(c2c1)-c1nc(N)ncc1CCC3,199.0
86,CHEMBL5181163,CC(C)(O)C#Cc1ccc2[nH]cc(-c3ccnc(N)n3)c2c1,1022.0
87,CHEMBL5279159,CC(C)(O)C#Cc1ccc2[nH]nc(-c3ccnc(N)n3)c2c1,185.0
88,CHEMBL5275588,Cn1nc(-c2ccnc(N)n2)c2cc(C#CC(C)(C)O)ncc21,352.0


Saves dataframe to CSV file

In [18]:
df3.to_csv('TTBK1_02_bioactivity_data_preprocessed.csv', index=False)

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**.

In [19]:
df4 = pd.read_csv('TTBK1_02_bioactivity_data_preprocessed.csv')

In [20]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [21]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL3745885,Cn1c(=O)c(S(=O)(=O)c2ccc(F)cc2F)cc2cnc(Nc3ccc4...,10000.0,inactive
1,CHEMBL4528881,CC1(N)CCC(Nc2ccn3ncnc(Nc4cc(O)cc(Br)c4)c23)CC1,120.0,active
2,CHEMBL278041,Oc1ccc(-c2nc(-c3ccc(F)cc3)c(-c3ccncc3)[nH]2)cc1,47200.0,inactive
3,CHEMBL4438748,CN(C)c1ccc(C(=O)Nc2cccc(NC(=O)COc3ccc4c(=O)cco...,10000.0,inactive
4,CHEMBL5193906,CC1(N)CCN(Cc2ccn3ncnc(Nc4cc(O)cc(Br)c4)c23)CC1,120.0,active
...,...,...,...,...
82,CHEMBL2334586,CC(C)(O)C#Cc1ccc2[nH]c3c(c2c1)-c1nc(N)ncc1CCC3,199.0,active
83,CHEMBL5181163,CC(C)(O)C#Cc1ccc2[nH]cc(-c3ccnc(N)n3)c2c1,1022.0,intermediate
84,CHEMBL5279159,CC(C)(O)C#Cc1ccc2[nH]nc(-c3ccnc(N)n3)c2c1,185.0,active
85,CHEMBL5275588,Cn1nc(-c2ccnc(N)n2)c2cc(C#CC(C)(C)O)ncc21,352.0,active


Saves dataframe to CSV file

In [22]:
df5.to_csv('TTBK1_03_bioactivity_data_curated.csv', index=False)

In [25]:
! zip TTBK1.zip *.csv

  adding: TTBK1_01_bioactivity_data_raw.csv (deflated 93%)
  adding: TTBK1_02_bioactivity_data_preprocessed.csv (deflated 74%)
  adding: TTBK1_03_bioactivity_data_curated.csv (deflated 76%)


In [26]:
! ls -l

total 156
-rw-r--r-- 1 root root  10816 Aug  6 07:30 acetylcholinesterase.zip
drwxr-xr-x 1 root root   4096 Jul 29 13:36 sample_data
-rw-r--r-- 1 root root 112011 Aug  6 07:28 TTBK1_01_bioactivity_data_raw.csv
-rw-r--r-- 1 root root   5471 Aug  6 07:29 TTBK1_02_bioactivity_data_preprocessed.csv
-rw-r--r-- 1 root root   6264 Aug  6 07:30 TTBK1_03_bioactivity_data_curated.csv
-rw-r--r-- 1 root root  10816 Aug  6 07:31 TTBK1.zip


---